In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# #Pyramid
# def batch_pyramid(images):
#     pyramid_list = []
#     for i in tqdm(range(len(images))):
#         img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         G = img.copy()
#         gp = [G]
#         for i in range(4):
#             G = cv2.pyrDown(G)
#             gp.append(G)
#         lp = [gp[4]]
#         for i in range(4,0,-1):
#             GE = cv2.pyrUp(gp[i])
#             (x,y) = gp[i-1].shape
#             L = cv2.subtract(gp[i-1],GE[:x,:y])
#             lp.append(L)
#         lp_1 = cv2.normalize(lp[1], lp[1],0,255,cv2.NORM_MINMAX)
#         lp_2 = cv2.normalize(lp[2], lp[2],0,255,cv2.NORM_MINMAX)
#         lp_out = [cv2.resize(lp_1, (128,128)),cv2.resize(lp_2, (128,128))]
#         pyramid_list.append((np.array(lp_out)).astype(np.uint8))
#     pyramid_list=np.array(pyramid_list,dtype = float32).reshape(
#         [images.shape[0],images.shape[1],images.shape[2],2])/255.0
    
#     print (pyramid_list.shape)
#     return pyramid_list

In [3]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [4]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

trainImage_pyramid = batch_pyramid(trainImage)
testImage_pyramid = batch_pyramid(testImage)
nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)
np.save('../trainImage_pyramid_L.npy', trainImage_pyramid)
np.save('../testImage_pyramid_L.npy', testImage_pyramid)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] trainImage_pyramid matrix: {:.2f}MB".format(
    (trainImage_pyramid.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_pyramid matrix: {:.4f}MB".format(
    (testImage_pyramid.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/4000
[INFO] processed 2000/4000
[INFO] processed 3000/4000
[INFO] processed 4000/4000
[INFO] processed 156/156
(4000, 128, 128, 3)


  3%|▎         | 5/156 [00:00<00:03, 44.77it/s]

(4000, 128, 128, 2)


100%|██████████| 156/156 [00:03<00:00, 49.82it/s]


(156, 128, 128, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.pyramid_dim = 2  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_pyramid = np.load('../trainImage_pyramid_L.npy')
        self.test_x_pyramid = np.load('../testImage_pyramid_L.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_pyramid, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_pyramid",x_pyramid.get_shape()) # 128, 128, 3 
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_pyramid,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_pyramid = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.pyramid_dim], 
                                name='x_pyramid')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_pyramid = self.train_x_pyramid[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_pyramid= shuffled_set_pyramid[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_pyramid: batch_x_pyramid,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_pyramid: batch_x_pyramid_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramidtest =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [6]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# [50/50] - ptime: 10.6408 loss: 0.00001457 acc: 0.72000 lr: 0.00059049
# Avg per epoch ptime: 11.01, total 50 epochs ptime: 576.86
#  [*] Training finished!
#  [*] Best Epoch:  10 , Accuracy:  0.75
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-10
#  [*] Finished testing Best Epoch: 10 , accuracy:  0.75 !

CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x2x32) [576, bytes: 2304]
cnn/co

Epoch: [ 3] [  18/  40] time: 32.1126, loss: 0.06197466
Epoch: [ 3] [  19/  40] time: 32.3788, loss: 0.03287204
Epoch: [ 3] [  20/  40] time: 32.6282, loss: 0.04513976
Epoch: [ 3] [  21/  40] time: 32.8853, loss: 0.05670438
Epoch: [ 3] [  22/  40] time: 33.1390, loss: 0.03479360
Epoch: [ 3] [  23/  40] time: 33.3936, loss: 0.01997443
Epoch: [ 3] [  24/  40] time: 33.6438, loss: 0.05966530
Epoch: [ 3] [  25/  40] time: 33.8930, loss: 0.04162209
Epoch: [ 3] [  26/  40] time: 34.1483, loss: 0.10198610
Epoch: [ 3] [  27/  40] time: 34.3977, loss: 0.07680871
Epoch: [ 3] [  28/  40] time: 34.6455, loss: 0.03881118
Epoch: [ 3] [  29/  40] time: 34.8939, loss: 0.03795028
Epoch: [ 3] [  30/  40] time: 35.1514, loss: 0.02760784
Epoch: [ 3] [  31/  40] time: 35.4040, loss: 0.02328650
Epoch: [ 3] [  32/  40] time: 35.6450, loss: 0.02760923
Epoch: [ 3] [  33/  40] time: 35.8923, loss: 0.05791844
Epoch: [ 3] [  34/  40] time: 36.1504, loss: 0.01300879
Epoch: [ 3] [  35/  40] time: 36.3998, loss: 0.0

Epoch: [ 7] [   0/  40] time: 74.1856, loss: 0.01155573
Epoch: [ 7] [   1/  40] time: 74.4353, loss: 0.00348218
Epoch: [ 7] [   2/  40] time: 74.6858, loss: 0.00359596
Epoch: [ 7] [   3/  40] time: 74.9422, loss: 0.01398062
Epoch: [ 7] [   4/  40] time: 75.1948, loss: 0.00705005
Epoch: [ 7] [   5/  40] time: 75.4501, loss: 0.00322538
Epoch: [ 7] [   6/  40] time: 75.6985, loss: 0.02328734
Epoch: [ 7] [   7/  40] time: 75.9527, loss: 0.00132199
Epoch: [ 7] [   8/  40] time: 76.2046, loss: 0.00747291
Epoch: [ 7] [   9/  40] time: 76.4558, loss: 0.00261334
Epoch: [ 7] [  10/  40] time: 76.7062, loss: 0.00362312
Epoch: [ 7] [  11/  40] time: 76.9634, loss: 0.00092502
Epoch: [ 7] [  12/  40] time: 77.2154, loss: 0.00284757
Epoch: [ 7] [  13/  40] time: 77.4648, loss: 0.00254669
Epoch: [ 7] [  14/  40] time: 77.7246, loss: 0.01097309
Epoch: [ 7] [  15/  40] time: 77.9817, loss: 0.00797208
Epoch: [ 7] [  16/  40] time: 78.2316, loss: 0.00505298
Epoch: [ 7] [  17/  40] time: 78.4828, loss: 0.0

Epoch: [10] [  22/  40] time: 114.7765, loss: 0.00332963
Epoch: [10] [  23/  40] time: 115.0261, loss: 0.00164649
Epoch: [10] [  24/  40] time: 115.2751, loss: 0.00034310
Epoch: [10] [  25/  40] time: 115.5333, loss: 0.01460745
Epoch: [10] [  26/  40] time: 115.7848, loss: 0.03537609
Epoch: [10] [  27/  40] time: 116.0331, loss: 0.00764632
Epoch: [10] [  28/  40] time: 116.2790, loss: 0.00049362
Epoch: [10] [  29/  40] time: 116.5296, loss: 0.00052990
Epoch: [10] [  30/  40] time: 116.7827, loss: 0.00128544
Epoch: [10] [  31/  40] time: 117.0315, loss: 0.06424870
Epoch: [10] [  32/  40] time: 117.2814, loss: 0.00099299
Epoch: [10] [  33/  40] time: 117.5336, loss: 0.00612686
Epoch: [10] [  34/  40] time: 117.7874, loss: 0.00402764
Epoch: [10] [  35/  40] time: 118.0394, loss: 0.00398076
Epoch: [10] [  36/  40] time: 118.2852, loss: 0.00223228
Epoch: [10] [  37/  40] time: 118.5374, loss: 0.00361915
Epoch: [10] [  38/  40] time: 118.7907, loss: 0.00036112
Epoch: [10] [  39/  40] time: 1

Epoch: [14] [   1/  40] time: 156.0721, loss: 0.00063327
Epoch: [14] [   2/  40] time: 156.3217, loss: 0.00001337
Epoch: [14] [   3/  40] time: 156.5753, loss: 0.00002368
Epoch: [14] [   4/  40] time: 156.8264, loss: 0.00067068
Epoch: [14] [   5/  40] time: 157.0739, loss: 0.00077422
Epoch: [14] [   6/  40] time: 157.3212, loss: 0.00002346
Epoch: [14] [   7/  40] time: 157.5784, loss: 0.00028840
Epoch: [14] [   8/  40] time: 157.8265, loss: 0.00123743
Epoch: [14] [   9/  40] time: 158.0764, loss: 0.00020181
Epoch: [14] [  10/  40] time: 158.3267, loss: 0.00019133
Epoch: [14] [  11/  40] time: 158.5830, loss: 0.00027273
Epoch: [14] [  12/  40] time: 158.8347, loss: 0.00020946
Epoch: [14] [  13/  40] time: 159.0820, loss: 0.00001589
Epoch: [14] [  14/  40] time: 159.3310, loss: 0.00005079
Epoch: [14] [  15/  40] time: 159.5895, loss: 0.00004020
Epoch: [14] [  16/  40] time: 159.8376, loss: 0.00275783
Epoch: [14] [  17/  40] time: 160.0837, loss: 0.00030005
Epoch: [14] [  18/  40] time: 1

...

Epoch: [35] [   1/  40] time: 399.8355, loss: 0.00000965
Epoch: [35] [   2/  40] time: 400.0936, loss: 0.00000325
Epoch: [35] [   3/  40] time: 400.3387, loss: 0.00001335
Epoch: [35] [   4/  40] time: 400.5862, loss: 0.00000370
Epoch: [35] [   5/  40] time: 400.8411, loss: 0.00000678
Epoch: [35] [   6/  40] time: 401.0839, loss: 0.00000094
Epoch: [35] [   7/  40] time: 401.3291, loss: 0.00012290
Epoch: [35] [   8/  40] time: 401.5855, loss: 0.00000035
Epoch: [35] [   9/  40] time: 401.8295, loss: 0.00008944
Epoch: [35] [  10/  40] time: 402.0773, loss: 0.00000507
Epoch: [35] [  11/  40] time: 402.3287, loss: 0.00006015
Epoch: [35] [  12/  40] time: 402.5743, loss: 0.00001480
Epoch: [35] [  13/  40] time: 402.8210, loss: 0.00001667
Epoch: [35] [  14/  40] time: 403.0652, loss: 0.00001041
Epoch: [35] [  15/  40] time: 403.3105, loss: 0.00000382
Epoch: [35] [  16/  40] time: 403.5621, loss: 0.00003517
Epoch: [35] [  17/  40] time: 403.8090, loss: 0.00003426
Epoch: [35] [  18/  40] time: 4

Epoch: [38] [  22/  40] time: 438.3554, loss: 0.00001432
Epoch: [38] [  23/  40] time: 438.6008, loss: 0.00002097
Epoch: [38] [  24/  40] time: 438.8582, loss: 0.00000161
Epoch: [38] [  25/  40] time: 439.1023, loss: 0.00000177
Epoch: [38] [  26/  40] time: 439.3483, loss: 0.00001036
Epoch: [38] [  27/  40] time: 439.6054, loss: 0.00000519
Epoch: [38] [  28/  40] time: 439.8573, loss: 0.00001269
Epoch: [38] [  29/  40] time: 440.1039, loss: 0.00000613
Epoch: [38] [  30/  40] time: 440.3508, loss: 0.00001424
Epoch: [38] [  31/  40] time: 440.5959, loss: 0.00001279
Epoch: [38] [  32/  40] time: 440.8454, loss: 0.00000302
Epoch: [38] [  33/  40] time: 441.0930, loss: 0.00000118
Epoch: [38] [  34/  40] time: 441.3377, loss: 0.00004004
Epoch: [38] [  35/  40] time: 441.5914, loss: 0.00001795
Epoch: [38] [  36/  40] time: 441.8429, loss: 0.00000145
Epoch: [38] [  37/  40] time: 442.0856, loss: 0.00000843
Epoch: [38] [  38/  40] time: 442.3437, loss: 0.00000278
Epoch: [38] [  39/  40] time: 4

Epoch: [42] [   1/  40] time: 477.7516, loss: 0.00000466
Epoch: [42] [   2/  40] time: 478.0028, loss: 0.00002304
Epoch: [42] [   3/  40] time: 478.2522, loss: 0.00000906
Epoch: [42] [   4/  40] time: 478.4982, loss: 0.00001631
Epoch: [42] [   5/  40] time: 478.7541, loss: 0.00000068
Epoch: [42] [   6/  40] time: 478.9992, loss: 0.00000355
Epoch: [42] [   7/  40] time: 479.2439, loss: 0.00006792
Epoch: [42] [   8/  40] time: 479.5037, loss: 0.00000799
Epoch: [42] [   9/  40] time: 479.7485, loss: 0.00000828
Epoch: [42] [  10/  40] time: 479.9951, loss: 0.00000362
Epoch: [42] [  11/  40] time: 480.2528, loss: 0.00000532
Epoch: [42] [  12/  40] time: 480.4986, loss: 0.00002523
Epoch: [42] [  13/  40] time: 480.7458, loss: 0.00001243
Epoch: [42] [  14/  40] time: 480.9910, loss: 0.00001663
Epoch: [42] [  15/  40] time: 481.2343, loss: 0.00000856
Epoch: [42] [  16/  40] time: 481.4917, loss: 0.00000024
Epoch: [42] [  17/  40] time: 481.7410, loss: 0.00003048
Epoch: [42] [  18/  40] time: 4

Epoch: [45] [  22/  40] time: 516.5980, loss: 0.00000300
Epoch: [45] [  23/  40] time: 516.8415, loss: 0.00009332
Epoch: [45] [  24/  40] time: 517.0944, loss: 0.00000440
Epoch: [45] [  25/  40] time: 517.3423, loss: 0.00000799
Epoch: [45] [  26/  40] time: 517.5893, loss: 0.00002820
Epoch: [45] [  27/  40] time: 517.8460, loss: 0.00000058
Epoch: [45] [  28/  40] time: 518.0891, loss: 0.00000237
Epoch: [45] [  29/  40] time: 518.3326, loss: 0.00000398
Epoch: [45] [  30/  40] time: 518.5885, loss: 0.00000251
Epoch: [45] [  31/  40] time: 518.8318, loss: 0.00000120
Epoch: [45] [  32/  40] time: 519.0801, loss: 0.00000074
Epoch: [45] [  33/  40] time: 519.3248, loss: 0.00001918
Epoch: [45] [  34/  40] time: 519.5703, loss: 0.00001449
Epoch: [45] [  35/  40] time: 519.8242, loss: 0.00000176
Epoch: [45] [  36/  40] time: 520.0691, loss: 0.00003714
Epoch: [45] [  37/  40] time: 520.3149, loss: 0.00002993
Epoch: [45] [  38/  40] time: 520.5732, loss: 0.00001285
Epoch: [45] [  39/  40] time: 5

Epoch: [49] [   1/  40] time: 555.8527, loss: 0.00000305
Epoch: [49] [   2/  40] time: 556.0975, loss: 0.00001014
Epoch: [49] [   3/  40] time: 556.3527, loss: 0.00000069
Epoch: [49] [   4/  40] time: 556.5996, loss: 0.00000320
Epoch: [49] [   5/  40] time: 556.8482, loss: 0.00000556
Epoch: [49] [   6/  40] time: 557.1026, loss: 0.00000354
Epoch: [49] [   7/  40] time: 557.3459, loss: 0.00000779
Epoch: [49] [   8/  40] time: 557.5955, loss: 0.00000518
Epoch: [49] [   9/  40] time: 557.8563, loss: 0.00005999
Epoch: [49] [  10/  40] time: 558.1007, loss: 0.00000146
Epoch: [49] [  11/  40] time: 558.3475, loss: 0.00000485
Epoch: [49] [  12/  40] time: 558.5943, loss: 0.00002216
Epoch: [49] [  13/  40] time: 558.8406, loss: 0.00000244
Epoch: [49] [  14/  40] time: 559.0924, loss: 0.00001120
Epoch: [49] [  15/  40] time: 559.3453, loss: 0.00000150
Epoch: [49] [  16/  40] time: 559.5901, loss: 0.00000267
Epoch: [49] [  17/  40] time: 559.8539, loss: 0.00001767
Epoch: [49] [  18/  40] time: 5

In [7]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放